In [38]:
#import albumentations
import cv2
import glob
import numpy as np
import pandas as pd
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import Sequential, optimizers
from keras.callbacks import ModelCheckpoint
from keras.layers import BatchNormalization, Activation, Conv2D, Dense, Dropout, Flatten, GlobalMaxPool2D, MaxPooling2D, Softmax
import keras
from skimage import io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import tensorflow as tf

In [39]:
a = pd.read_csv('landmark-recognition-challenge/train.csv')
OUTPUT_DIR  = 'output'


In [40]:
t = glob.glob('output/' + '*.jpg')


In [41]:
arr_list = []
lost_files = 0
lost_names = []
for file in os.listdir('output/'):
    try:
        img = io.imread('output/' + file)
        input_arr = np.array([img])  # Convert single image to a batch.
        inpur_arr = np.expand_dims(input_arr, axis=0)  # or axis=1
        arr_list.append(input_arr)
    except:
        print('could not get file')
        lost_files+=1
        lost_names.append(file)


could not get file


In [42]:
id_list = []
for file in os.listdir('output'):
    file_name, ext = os.path.splitext(file)
    id_list.append(file_name)

In [43]:
train_data = a[a.id.isin(id_list)]
train_data = train_data[~train_data.id.isin(lost_names)]

In [44]:
test_path = 'output/'
test_image_files = glob.glob(test_path + '*.jpg')
test_image_ids = [image_file.replace(
    '.jpg', '').replace(test_path, '') for image_file in test_image_files]


In [73]:
test_image_ids.remove(os.path.splitext(*lost_names)[0])

In [75]:
label_encoder = LabelEncoder()
train_info = a[a.id.isin(test_image_ids)]
labels = label_encoder.fit_transform(train_info['landmark_id'].values)
one_hot_encoder = OneHotEncoder(sparse=True, categories=train_data['landmark_id'].nunique())
train_info['labels'] = label_encoder.fit_transform(train_info['landmark_id'].values)
e = train_info['labels'].values.reshape(-1,1)
onehot = pd.get_dummies(train_info['labels'])

/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [76]:
Y = onehot
#Y = Y[:-1]
X = arr_list

In [77]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2


In [78]:
input_shape = X_train[0].shape[1], X_train[0].shape[2], X_train[0].shape[3] 

In [16]:
#gpu_options = tf.GPUOptions(allow_growth=True)
#gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
#custom_config=tf.compat.v1.ConfigProto(log_device_placement=True, allow_soft_placement=True)
#custom_config.gpu_options.allocator_type = 'BFC'
#custom_config.gpu_options.per_process_gpu_memory_fraction = 0.90

#session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))


In [79]:
modelBN = Sequential()
modelBN.add(Conv2D(filters=16, kernel_size=4,strides=(2, 2), padding='same', use_bias=True, input_shape=input_shape))
modelBN.add(Activation("relu"))
modelBN.add(BatchNormalization())
modelBN.add(MaxPooling2D(pool_size=2))

modelBN.add(Conv2D(filters=32, kernel_size=3, strides=(2, 2),padding='same', use_bias=True))
modelBN.add(Activation("relu"))
modelBN.add(BatchNormalization())
modelBN.add(MaxPooling2D(pool_size=2))

modelBN.add(Conv2D(filters=64, kernel_size=3, strides=(2, 2), padding='same', use_bias=True))
modelBN.add(Activation("relu"))
modelBN.add(BatchNormalization())
modelBN.add(MaxPooling2D(pool_size=2))

modelBN.add(Conv2D(filters=128, kernel_size=2, strides=(4, 4), padding='same', use_bias=True))
modelBN.add(Activation("relu"))
modelBN.add(BatchNormalization())
modelBN.add(MaxPooling2D(pool_size=2))

modelBN.add(Flatten())
modelBN.add(Dense(256, activation='relu'))
modelBN.add(Dropout(0.3))
modelBN.add(Dense(train_data['landmark_id'].nunique(), activation='softmax'))

modelBN.summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 600, 600, 16)      784       
_________________________________________________________________
activation_15 (Activation)   (None, 600, 600, 16)      0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 600, 600, 16)      64        
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 300, 300, 16)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 150, 150, 32)      4640      
_________________________________________________________________
activation_16 (Activation)   (None, 150, 150, 32)      0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 150, 150, 32)     

In [80]:
adam = optimizers.Adam(lr=0.02)
modelBN.compile(optimizer='adamax', loss='categorical_crossentropy', metrics=['accuracy'])


NameError: name 'optimizers' is not defined

In [ ]:
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_BatchNorm.hdf5', 
                               verbose=1, save_best_only=True)

epochs_batch = 25

hist_BN = modelBN.fit(np.squeeze(X_train), Y_train, validation_data=(np.squeeze(X_test), Y_test), 
epochs=epochs_batch, batch_size=32, callbacks=[checkpointer], verbose=1)


Train on 3190 samples, validate on 1064 samples
Epoch 1/25
3190/3190 [==============================] - 109s 34ms/step - loss: 7.4748 - accuracy: 0.0414 - val_loss: 7.4403 - val_accuracy: 0.0367

Epoch 00001: val_loss improved from inf to 7.44031, saving model to saved_models/weights.best.from_BatchNorm.hdf5
Epoch 2/25
3190/3190 [==============================] - 107s 34ms/step - loss: 6.6801 - accuracy: 0.0411 - val_loss: 7.3618 - val_accuracy: 0.0348

Epoch 00002: val_loss improved from 7.44031 to 7.36176, saving model to saved_models/weights.best.from_BatchNorm.hdf5
Epoch 3/25
3190/3190 [==============================] - 107s 34ms/step - loss: 6.2159 - accuracy: 0.0498 - val_loss: 7.5388 - val_accuracy: 0.0404

Epoch 00003: val_loss did not improve from 7.36176
Epoch 4/25
3190/3190 [==============================] - 108s 34ms/step - loss: 5.7904 - accuracy: 0.0545 - val_loss: 7.8569 - val_accuracy: 0.0423

Epoch 00004: val_loss did not improve from 7.36176
Epoch 5/25
3190/3190 [====

In [31]:
# import tensorflow as tf
# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:1 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:2 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:3 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:4 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:5 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:6 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:7 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla K80, pci bus id: 0000:00:17.0, compute capability: 3.7
/job:localhost/replica:0/task:0/device:GPU:1 -> device: 1, name: Tesla K80, pci bus id: 0000:00:18.0, compute capability: 3.7
/job:localhost/replica:0/task:0/device:GPU:2 -> device: 2

In [ ]:
modelBN.load_weights('saved_models/weights.best.from_BatchNorm.hdf5')

landmark_pred = [np.argmax(modelBN.predict(np.expand_dims(tensor, axis=0))) for tensor in X_val]
test_accuracy = np.sum(np.array(landmark_pred) == np.argmax(Y_val, axis=1)) / len(landmark_pred)
print('Test accuracy:', test_accuracy)


In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=4.,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.5,
    channel_shift_range=25,
    horizontal_flip=True,
    fill_mode='nearest')


valid_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(X_train, Y_train, batch_size= 128)
valid_generator = valid_datagen.flow(X_val, Y_val, batch_size= 128)
epochs_aug = 50

hist_aug = modelBN.fit_generator(train_generator, steps_per_epoch=6480//128, epochs=epochs_aug,
                    validation_data=valid_generator, validation_steps=805//128,
                    callbacks=[checkpointer], verbose=1)


In [ ]:

transforms_672 = albumentations.Compose([
    albumentations.Resize(672, 672),
    albumentations.Normalize()
])

transforms_768 = albumentations.Compose([
    albumentations.Resize(768, 768),
    albumentations.Normalize()
])
transforms_512 = albumentations.Compose([
    albumentations.Resize(512, 512),
    albumentations.Normalize()
])

for row in os.listdir('outputs'):
    image = cv2.imread(row)
    image = image[:, :, ::-1]

    res0 = self.transform672(image=image)
    image0 = res0['image'].astype(np.float32)
    image0 = image0.transpose(2, 0, 1)        

    res1 = self.transform768(image=image)
    image1 = res1['image'].astype(np.float32)
    image1 = image1.transpose(2, 0, 1)    

    res3 = self.transform512(image=image)
    image3 = res3['image'].astype(np.float32)        
    image3 = image3.transpose(2, 0, 1)   
